In [ ]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd
import anndata as ad
import loompy
import matplotlib as plt
import igraph

In [ ]:
# import CellRank kernels and estimators
from cellrank.kernels import ConnectivityKernel
from cellrank.kernels import CytoTRACEKernel
from cellrank.kernels import VelocityKernel
from cellrank.estimators import GPCCA

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', transparent=False, facecolor='white', dpi=200, dpi_save=500, frameon=False)
cr.settings.verbosity = 2

In [ ]:
adata = sc.read_h5ad("F:/Gekko_Gecko/SingleCell_Analysis/SC_Analysis_Seurat/allRT.integrated_celltypes_filt.h5ad")
adata

In [ ]:
ldata3 = scv.read('F:/Gekko_Gecko/SingleCell_Analysis/velodata/RT_16dpa.loom', cache=True)
ldata4 = scv.read('F:/Gekko_Gecko/SingleCell_Analysis/velodata/RT_20dpa.loom', cache=True)
ldata5 = scv.read('F:/Gekko_Gecko/SingleCell_Analysis/velodata/RT_24dpa.loom', cache=True)
ldata6 = scv.read('F:/Gekko_Gecko/SingleCell_Analysis/velodata/RT_28dpa.loom', cache=True)

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata3.obs.index.tolist()]
barcodes = ['RT_16dpa_' + bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata3.obs.index = barcodes
ldata3.obs.index

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata4.obs.index.tolist()]
barcodes = ['RT_20dpa_' + bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata4.obs.index = barcodes
ldata4.obs.index

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata5.obs.index.tolist()]
barcodes = ['RT_24dpa_' + bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata5.obs.index = barcodes
ldata5.obs.index

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata6.obs.index.tolist()]
barcodes = ['RT_28dpa_' + bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata6.obs.index = barcodes
ldata6.obs.index

In [ ]:
# make variable names unique
ldata3.var_names_make_unique()
ldata4.var_names_make_unique()
ldata5.var_names_make_unique()
ldata6.var_names_make_unique()

In [ ]:
adata.obs.index

In [ ]:
# concatenate the six loom
ldata_RT = ldata3.concatenate([ldata4, ldata5, ldata6])

In [ ]:
#clean the sample barcodes and save the sample name to sampel_batch
scv.utils.clean_obs_names(adata)
scv.utils.clean_obs_names(ldata_RT)

In [ ]:
adata_merge = scv.utils.merge(adata, ldata_RT)
adata_merge

In [ ]:
adata_merge.__dict__['_raw'].__dict__['_var'] = adata_merge.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
adata_merge.write('F:/Gekko_Gecko/SingleCell_Analysis/SC_Analysis_Seurat/adataveloraw_RT.h5ad')

In [ ]:
sc.pl.umap(adata_merge, color='celltypes', frameon=False, legend_loc='right margin', title='', save=None)

In [ ]:
scv.pl.proportions(adata_merge, groupby='celltypes', layers=None, highlight='unspliced', add_labels_pie=True, add_labels_bar=True, fontsize=10, figsize=(15, 3), dpi=500, use_raw=True, show=True, save="RT_proportioncell.png")

In [ ]:
sc.pp.neighbors(adata_merge)

In [ ]:
sc.tl.draw_graph(adata_merge, layout='fr')
sc.tl.draw_graph(adata_merge, layout='drl')

In [ ]:
adata_merge.obsm

In [ ]:
sc.pl.embedding(
    adata_merge,
    basis="draw_graph_drl",
    color=["orig.ident", "celltypes"],
    color_map="gnuplot", save="RT_force-directed_drl.png",
)

In [ ]:
sc.pl.embedding(
    adata_merge,
    basis="draw_graph_fr",
    color=["orig.ident", "celltypes"],
    color_map="gnuplot", save="RT_force-directed_fr.png",
)

In [ ]:
scv.pp.filter_and_normalize(adata_merge)

In [ ]:
scv.pp.moments(adata_merge)

In [ ]:
ctk = CytoTRACEKernel(adata_merge).compute_cytotrace()

In [ ]:
scv.pl.scatter(
    adata_merge,
    c=["ct_pseudotime","ct_score","ct_num_exp_genes" ],
    basis="draw_graph_drl",
    legend_loc="right",
    color_map="gnuplot2", save="RT_force-directed_ctpseudotime.png",
)

In [ ]:
scv.pl.scatter(
    adata_merge,
    c=["ct_pseudotime","ct_score","ct_num_exp_genes"],
    basis="umap",
    legend_loc="right",
    color_map="gnuplot2", save="RT_umap_ctpseudotime.png",
)

In [ ]:
sc.pl.violin(adata_merge, keys=["ct_pseudotime","ct_score","ct_num_exp_genes"], groupby="orig.ident", rotation=90, save="RT_cellrank_ctpseudotime_stages.png")

In [ ]:
sc.pl.violin(adata_merge, keys=["ct_pseudotime","ct_score","ct_num_exp_genes"], groupby="celltypes", rotation=90, save="RT_cellrank_ctpseudotime_celltype.png")

In [ ]:
ctk.compute_transition_matrix(threshold_scheme="hard",) #nu=0.5

In [ ]:
ctk.plot_projection(basis="draw_graph_drl", color=["celltypes","orig.ident"], title="velocity stream of regenerating tail", 
    save="RT_embedstream_drl_celltype_cytokernel1.png")

In [ ]:
scv.pl.velocity_embedding_stream(
    adata_merge, 
   color=["celltypes"], 
    vkey="T_fwd", basis="draw_graph_drl", legend_loc="right", title="velocity stream of regenerating tail", 
    save="RT_embedstream_drl_celltype_cytokernel.png")

In [ ]:
scv.pl.velocity_embedding_stream(
    adata_merge, color="orig.ident", vkey="T_fwd", basis="draw_graph_drl",
legend_loc="right", title="velocity stream of regenerating tail",
  save="RT_embedstream_drl_stage_cytokernel.png")

In [ ]:
ctk.plot_projection(basis="umap", color=["celltypes", "orig.ident"], title="velocity stream of regenerating tail", 
    save="RT_embedstream_umap_celltype_cytokernel1.png")

In [ ]:
scv.pl.velocity_embedding_stream(
    adata_merge, color="orig.ident", vkey="T_fwd", basis="umap",
legend_loc="right", title="velocity stream of regenerating tail",
  save="RT_embedstream_umap_celltypes_cytokernel.png")

In [ ]:
scv.pl.velocity_embedding_stream(
    adata_merge, color="celltypes", vkey="T_fwd", basis="umap",
legend_loc="right", title="velocity stream of regenerating tail",
  save="RT_embedstream_umap_celltypes_cytokernel.png")

In [ ]:
ctk.plot_random_walks(
    n_sims=15,
    start_ixs={"orig.ident": "RT_16dpa"},
    basis="draw_graph_drl",
    color="orig.ident",
    legend_loc="right",
    seed=1, save="RT_randomwalk_drl_stage_cytokernel.png", title="random walk simulation on regenerating tail",
)

In [ ]:
ctk.plot_random_walks(
    n_sims=15,
    start_ixs={"orig.ident": "RT_16dpa"},
    basis="umap",
    color="celltypes",
    legend_loc="right",
    seed=1, title="random walk simulation on regenerating tail", save="RT_randomwalk_celltype_cytokernel.png",
)

In [ ]:
scv.tl.recover_dynamics(adata_merge, n_jobs=16)

In [ ]:
scv.tl.velocity(adata_merge, mode="dynamical")

In [ ]:
vk = cr.kernels.VelocityKernel(adata_merge)

In [ ]:
vk.compute_transition_matrix()

In [ ]:
vk.plot_projection(basis="draw_graph_drl", color="celltypes")

In [ ]:
ck = cr.kernels.ConnectivityKernel(adata_merge)
ck.compute_transition_matrix()

comb_kernel = 0.6 * vk + 0.4 * ck

In [ ]:
ck.plot_projection(basis="draw_graph_drl", color="celltypes")

In [ ]:
g_fwd = GPCCA(ctk)
print(g_fwd)

In [ ]:
g_fwd.compute_schur(n_components=20)
g_fwd.plot_spectrum(real_only=True)

In [ ]:
g_fwd.compute_macrostates(n_states=11, cluster_key="celltypes")

In [ ]:
g_fwd.plot_macrostates( 
    which="all", discrete=True, legend_loc="right", size=100, basis="draw_graph_drl", title="potential terminal macrostates on regenerating tail",
    save="RT_macrostate_drl_celltype_cytokernel.png",
)

In [ ]:
g_fwd.plot_coarse_T(title="coarse-grained transition matrix of terminal macrostates", 
                    save="RT_macrostatecoarse_celltype_cytokernel.png")

In [ ]:
g_fwd.plot_macrostate_composition(key="celltypes", save="RT_macrostatecomposition_celltype_cytokernel.png")

In [ ]:
g_fwd.compute_macrostates(n_states=11, cluster_key="orig.ident")
g_fwd.plot_macrostates(
    which="all", discrete=True, legend_loc="right", size=100, basis="draw_graph_drl", title="potential terminal macrostates on regenerating tail",
    save="RT_macrostate_stage_cytokernel.png"
)

In [ ]:
g_fwd.plot_coarse_T(title="coarse-grained transition matrix of terminal macrostates", save="RT_macrostatecoarse_stage_cytokernel.png")

In [ ]:
g_fwd.compute_macrostates(n_states=11, cluster_key="celltypes")
g_fwd.plot_macrostates(
    which="all", discrete=True, legend_loc="right", size=100, basis="draw_graph_drl", title="potential terminal macrostates on regenerating tail",
    save="RT_macrostate_celltype_cytokernel.png",
)

In [ ]:
g_fwd.plot_coarse_T()

In [ ]:
g_fwd.compute_macrostates(n_states=11, cluster_key="celltypes")
g_fwd.predict_terminal_states()

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_drl", legend_loc="right", title= "terminal states prediction", save="RT_terminalstate_celltype_cytokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_drl", 
                       legend_loc="right", title= "terminal states memberships",
                       save="RT_terminalstate_celltype_cytokernel2.png", discrete=False)

In [ ]:
g_fwd.predict_initial_states(allow_overlap=True)

In [ ]:
g_fwd.plot_macrostates(which="initial", same_plot=True, basis="draw_graph_drl", 
                       legend_loc="right", title= "initial states prediction",
                       save="RT_initialstate_celltype_cytokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="initial", same_plot=True, basis="draw_graph_drl", 
                       legend_loc="right", title= "initial states prediction",
                       save="RT_initialstate_celltype_cytokernel2.png", discrete=False)

In [ ]:
g_fwd.compute_macrostates(n_states=11, cluster_key="celltypes")
g_fwd.set_terminal_states(states=["Myoblasts","Erythroid cells", "Keratinocytes","Myocytes","Chondrocytes"])

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_drl", 
                       legend_loc="right", title= "assigned terminal states", 
                           save="RT_assignedterminalstate_celltype_cytokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_drl", 
                       legend_loc="right", title= "assigned terminal states memberships", discrete=False , 
                           save="RT_assignedterminalstate_celltype_cytokernel2.png")

In [ ]:
g_fwd.compute_fate_probabilities()
g_fwd.plot_fate_probabilities(same_plot=False, size=100, basis="draw_graph_drl", ncols=3, 
                                    save="RT_absorbfate_celltype_cytokernel.png",
)

In [ ]:
Myoblasts_drivers = g_fwd.compute_lineage_drivers(lineages="Myoblasts", return_drivers=True)
Myoblasts_drivers.sort_values(by="Myoblasts_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Myoblasts", 
                           basis="draw_graph_drl", 
                           n_genes=12, ncols=3, 
                           save="RT_lineagedriver_Myoblasts_cytokernel.png")

In [ ]:
Chondro_drivers = g_fwd.compute_lineage_drivers(lineages="Chondrocytes", return_drivers=True)
Chondro_drivers.sort_values(by="Chondrocytes_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Chondrocytes", 
                           basis="draw_graph_drl", 
                           n_genes=12, ncols=3, 
                           save="RT_lineagedriver_Chondrocytes_cytokernel.png")

In [ ]:
myocytes_drivers = g_fwd.compute_lineage_drivers(lineages="Myocytes", return_drivers=True)
myocytes_drivers.sort_values(by="Myocytes_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Myocytes", 
                           basis="draw_graph_drl", 
                           n_genes=12, ncols=3, 
                           save="RT_lineagedriver_Myocytes_cytokernel.png")

In [ ]:
Kera_drivers = g_fwd.compute_lineage_drivers(lineages="Keratinocytes", return_drivers=True)
Kera_drivers.sort_values(by="Keratinocytes_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Keratinocytes", 
                           basis="draw_graph_drl", 
                           n_genes=12, ncols=3, 
                           save="RT_lineagedriver_Keratinocytes_cytokernel.png")

In [ ]:
Ery_drivers = g_fwd.compute_lineage_drivers(lineages="Erythroid cells", return_drivers=True)
Ery_drivers.sort_values(by="Erythroid cells_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Erythroid cells", 
                           basis="draw_graph_drl", 
                           n_genes=12, ncols=3, 
                           save="RT_lineagedriver_Erythroid_cells_cytokernel.png")

In [ ]:
cr.pl.circular_projection(adata_merge, keys=["kl_divergence","celltypes"], figsize=[15,5] ,
                          legend_loc="right", ncols=2,
                          save="RT_circularprojection_cytokernel.png")

In [ ]:
### compute initial state
ctk2 = CytoTRACEKernel(adata_merge, backward=True, check_connectivity=True)

In [ ]:
ctk2.compute_transition_matrix(threshold_scheme="hard",) #nu=0.5)

In [ ]:
ctk2.plot_projection(basis="draw_graph_drl", color=["celltypes","orig.ident"], 
                     title="backward velocity stream of regenerating tail", 
                     save="RT_embedstream_drl_celltype_bwd_cytokernel.png")

In [ ]:
g_bwd = GPCCA(ctk2)
print(g_bwd)

In [ ]:
g_bwd.compute_schur(n_components=20)
g_bwd.plot_spectrum(real_only=True)

In [ ]:
g_bwd.compute_macrostates(n_states=7, cluster_key="celltypes")
g_bwd.plot_macrostates( 
    which="all", discrete=True, legend_loc="right", size=100, basis="draw_graph_drl", title="potential initial macrostates on regenerating tail",
    save="RT_macrostate_init_stage_celltype_cytokernel.png",
)

In [ ]:
g_bwd.plot_coarse_T(title="coarse-grained transition matrix of initial macrostates", save="RT_macrostatecoarse_celltype_bwd_cytokernel.png")

In [ ]:
g_bwd.compute_macrostates(n_states=7, cluster_key="orig.ident")
g_bwd.plot_macrostates(
     which="all", discrete=True, legend_loc="right", size=100, basis="draw_graph_drl", title="potential initial macrostates on regenerating tail",
    save="RT_macrostate_init_stages_cytokernel.png",
)

In [ ]:
g_bwd.compute_macrostates(n_states=7, cluster_key="celltypes")
g_bwd.predict_terminal_states()

In [ ]:
g_bwd.plot_macrostates(
     which="terminal", discrete=True, legend_loc="right", size=100, basis="draw_graph_drl", title="initial macrostates on regenerating tail",
    save="RT_initialstate_celltype_cytokernel.png",
)

In [ ]:
g_bwd.compute_fate_probabilities()
g_bwd.plot_fate_probabilities(same_plot=False, size=50, basis="draw_graph_drl", ncols=3,
                                    save="RT_fateprob_initialstate_cytokernel.png")

In [ ]:
g_bwd.set_terminal_states(states=["Stromal cells","Keratinocytes","Fibroblasts"])

In [ ]:
scv.tl.recover_dynamics(adata_merge)

In [ ]:
scv.tl.recover_latent_time(
    adata_merge, root_key="term_states_bwd_probs", end_key="term_states_fwd_probs"
)

In [ ]:
scv.tl.paga(
    adata_merge,
    groups="celltypes",
    root_key="term_states_bwd_probs",
    end_key="term_states_fwd_probs",
    use_time_prior="velocity_pseudotime",
)

In [ ]:
cr.pl.aggregate_fate_probabilities(adata_merge, 
                                         mode="paga_pie", 
                                         backward=False, lineages=None, 
                                         cluster_key='celltypes', clusters=None, 
                                         basis="draw_graph_drl", cbar=True, 
                                         ncols=None, sharey=False, fmt='0.2f', xrot=90, 
                                         legend_kwargs={'loc':  "top right out"},
                                         title="directed PAGA",
                                         save="RT_directed_PAGA_cytokernel.png")

In [ ]:
cr.pl.aggregate_fate_probabilities(adata_merge, 
                                         mode="paga_pie", 
                                         backward=False, lineages=None, 
                                         cluster_key='celltypes', clusters=None, 
                                         basis="draw_graph_drl", cbar=True, 
                                         ncols=None, sharey=False, fmt='0.2f', xrot=90, 
                                         legend_kwargs={'loc':  "top right out"},
                                         title="directed PAGA",
                                         save="RT_directed_PAGA_cytokernel.png")

In [ ]:
cr.pl.aggregate_fate_probabilities(
    adata_merge,
    mode="violin",
    lineages=["Myoblasts","Chondrocytes","Erythroid cells", "Keratinocytes", "Myocytes"],
    cluster_key="celltypes", ncols=2, #figsize=[5,15] # xrot=45, 
    save="RT_fate_probs_violoin_cytokernel.png"
)

In [ ]:
cr.pl.aggregate_fate_probabilities(
    adata_merge,
    mode="heatmap",
    lineages=["Myoblasts","Chondrocytes","Erythroid cells", "Keratinocytes", "Myocytes"],
    cluster_key="celltypes", ncols=3, #figsize=[5,15] # xrot=45, 
    save="RT_fate_probs_heatmap_cytokernel.png",
)

In [ ]:
cr.pl.circular_projection(adata_merge, keys="celltypes", legend_loc="right")

In [ ]:
cr.tl.terminal_states(
    adata_merge,
    cluster_key="celltypes",
    weight_connectivities=0.2,
    n_states=3,
    softmax_scale=4,
    show_progress_bar=False,
)
cr.tl.lineages(adata_merge)

model = cr.ul.models.GAM(adata_merge)

In [ ]:
model = cr.models.GAM(adata_merge)

In [ ]:
# obtain some genes from the same cluster, sort by mean expression
mid_peak_genes = (
    gdata[gdata.obs["clusters"] == "5"]
    .obs.sort_values("means", ascending=False)
    .head(8)
    .index
)

# plot
cr.pl.gene_trends(
    adata,
    model=model,
    lineages="Beta",
    cell_color="clusters",
    data_key="magic_imputed_data",
    genes=["Pax4"] + list(mid_peak_genes),
    same_plot=True,
    ncols=3,
    time_key="palantir_pseudotime",
    hide_cells=False,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
cr.pl.gene_trends(
    adata_merge,
    model,
    ["SHH", "MYOG"],
    data_key="Ms",
    time_key="ct_pseudotime",
    show_progress_bar=False,
)

In [ ]:
cr.pl.gene_trends(
    adata_merge,
    model,
    ["COL2A1", "MYOG", "MYOD1","LOC125439306"],
    data_key="Ms",
    same_plot=True,
    hide_cells=True,
    time_key="ct_pseudotime",
    show_progress_bar=False,
)

In [ ]:
pd.__version__